In [2]:
import numpy as np
import pandas as pd
from sklearn import *
from textblob import TextBlob

zpolarity = {0:'zero',1:'one',2:'two',3:'three',4:'four',5:'five',6:'six',7:'seven',8:'eight',9:'nine',10:'ten'}
zsign = {-1:'negative',  0.: 'neutral', 1:'positive'}

DATA_DIR = '~/data/toxic/data/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
#sub1 = pd.read_csv('../input/lstm-with-bn-nb-svm-lr-on-conv-ai-lb-0-041/submission_ensemble.csv')

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This mod

In [9]:
temp = train.comment_text[1]
temp

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [12]:
po = TextBlob(temp).sentiment.polarity
po

0.2875

In [14]:
np.sign(po), np.abs(po)

(1.0, 0.2875)

In [7]:
coly = [c for c in train.columns if c not in ['id','comment_text']]
y = train[coly]
tid = test['id'].values

train['polarity'] = train['comment_text'].map(ambda x: int(TextBlob(x).sentiment.polarity * 10))
test['polarity'] = test['comment_text'].map(lambda x: int(TextBlob(x).sentiment.polarity * 10))

In [8]:
train.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,polarity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,2


In [15]:
train['comment_text'] = train.apply(lambda r: str(r['comment_text']) + ' polarity' +  zsign[np.sign(r['polarity'])] + zpolarity[np.abs(r['polarity'])], axis=1)
test['comment_text'] = test.apply(lambda r: str(r['comment_text']) + ' polarity' +  zsign[np.sign(r['polarity'])] + zpolarity[np.abs(r['polarity'])], axis=1)

In [16]:
train.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,polarity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,2


In [17]:
train.comment_text[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27 polaritypositiveone"

In [18]:
df = pd.concat([train['comment_text'], test['comment_text']], axis=0)
df = df.fillna("unknown")
nrow = train.shape[0]

tfidf = feature_extraction.text.TfidfVectorizer(stop_words='english', max_features=800000)
data = tfidf.fit_transform(df)

In [20]:
data.shape, train.shape[0] + test.shape[0]

((312735, 353673), 312735)

In [21]:
model = ensemble.ExtraTreesClassifier(n_jobs=-1, random_state=3) 
model.fit(data[:nrow], y)
print(1- model.score(data[:nrow], y)) # acc on training set

0.0008585519925299323


In [22]:
sub2 = model.predict_proba(data[nrow:]) # decision tree vote. default # of trees: 10, so prediction will be 
                                        # something like: predict 0: 3/10, predict 1: 7/10

In [25]:
len(sub2), len(sub2[0])

(6, 153164)

In [29]:
for c in sub2[0]:
    print(c, c[1]) # c[1] is the prob of predicting 1
    break

[0.3 0.7] 0.7


In [30]:
sub2 = pd.DataFrame([[c[1] for c in sub2[row]] for row in range(len(sub2))]).T
sub2.columns = coly
sub2['id'] = tid

In [35]:
sub2.tail(10)

,toxic,severe_toxic,obscene,threat,insult,identity_hate,id
153154,0.5,0.0,0.0,0.0,0.0,0.0,fffa8a11c4378854
153155,0.5,0.0,0.4,0.0,0.4,0.0,fffac2a094c8e0e2
153156,0.1,0.0,0.0,0.0,0.1,0.0,fffb5451268fb5ba
153157,0.0,0.0,0.0,0.0,0.0,0.0,fffc2b34bbe61c8d
153158,0.4,0.0,0.2,0.0,0.3,0.1,fffc489742ffe69b
153159,0.2,0.0,0.1,0.0,0.0,0.0,fffcd0960ee309b5
153160,0.0,0.0,0.0,0.0,0.0,0.0,fffd7a9a6eb32c16
153161,0.0,0.0,0.0,0.0,0.0,0.0,fffda9e8d6fafa9e
153162,0.1,0.0,0.0,0.0,0.0,0.0,fffe8f1340a79fc2
153163,0.9,0.0,0.6,0.0,0.4,0.0,ffffce3fb183ee80


In [43]:
0+(1e2), 1+(-1e2), -1e2

(100.0, -99.0, -100.0)

In [44]:
for c in coly:
    sub2[c] = sub2[c].clip(0+1e12, 1-1e12) # ???

In [45]:
sub2.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'id'],
      dtype='object')

In [47]:
#blend 1
sub2.columns = [x+'_' if x not in ['id'] else x for x in sub2.columns]
sub2.columns

Index(['toxic_', 'severe_toxic_', 'obscene_', 'threat_', 'insult_',
       'identity_hate_', 'id'],
      dtype='object')

In [51]:
sub2.columns = [x[:-1] if x not in ['id'] else x for x in sub2.columns]

In [52]:
sub2.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'id'],
      dtype='object')

In [53]:
sub2.shape

(153164, 7)

In [55]:
sub2 = sub2[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
sub2.head(2)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.7,0.1,0.6,0.0,0.7,0.2
1,0000247867823ef7,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
sub2.to_csv('/home/kai/data/shiyi/toxic/submissions/sub_polarity.csv')

In [ ]:
blend = pd.merge(sub1, sub2, how='left', on='id')
for c in coly:
    blend[c] = blend[c] * 0.8 + blend[c+'_'] * 0.2
    blend[c] = blend[c].clip(0+1e12, 1-1e12)
blend = blend[sub1.columns]

#blend 2
sub2 = blend[:]
sub2.columns = [x+'_' if x not in ['id'] else x for x in sub2.columns]
blend = pd.merge(sub1, sub2, how='left', on='id')
for c in coly:
    blend[c] = np.sqrt(blend[c] * blend[c+'_'])
    blend[c] = blend[c].clip(0+1e12, 1-1e12)
blend = blend[sub1.columns]
blend.to_csv('submission.csv', index=False)

In [59]:
caches = []

In [63]:

xt = np.arange(12).reshape(3,4)
xt

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [65]:
a_prev = np.arange(20).reshape(5,4)
a_prev

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [67]:
c = np.concatenate((a_prev, xt))
c

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [68]:
c[:5,:]

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [69]:
c[5:,:]

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [70]:
d = np.zeros((a_prev.shape[0]+xt.shape[0], a_prev.shape[1]))
d

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [71]:
d[:5,:] = a_prev
d

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.],
       [16., 17., 18., 19.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [72]:
d[5:,:] = xt
d

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.],
       [16., 17., 18., 19.],
       [ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [75]:
x = np.arange(24).reshape(3,2,4)
x

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [77]:
x[:,:,0].shape

(3, 2)